## Preparación de datos para sistema recomendador proxy

En este *notebook* prepararemos los datos necesarios para evaluar un sistema recomendador externo con la librería `Elliot`. Se usarán los datasets de libros, usuarios y ratings.

### Creación de datasets de entrenamiento, validación y test

In [1]:
import os
import pandas as pd

# Ruta de datasets procesados
ready_path = os.path.join(os.getcwd(), "../..", "datasets", "ready")

# Lectura del dataset de ratings procesado
col_types = {
    'user_id': 'int32',
    'book_id': 'int32',
    'rating': 'float32',
}
ratings_df = pd.read_csv(ready_path + "/ratings.csv", dtype=col_types)
print("# de ratings:", ratings_df.shape[0])

# de ratings: 5515602


In [2]:
# Ruta de datasets en bruto
raw_path = os.path.join(os.getcwd(), "../..", "datasets", "raw")

# Lectura del dataset de libros en bruto (coincidirá con el procesado después de limpiarlo)
books_df = pd.DataFrame(pd.read_pickle(raw_path + "/books_raw.pkl"))
print("# de libros:", books_df.shape[0])

# de libros: 9467


In [3]:
# IDs de usuarios
user_ids = ratings_df['user_id'].unique()
print("# de usuarios:", len(user_ids))

# de usuarios: 52371


Debido a la gran cantidad de usuarios, seleccionamos 10000 de ellos aleatoriamente para un entrenamiento más rápido.

In [4]:
# Selección de num_users usuarios aleatorios
import random

random.seed(42)
num_users = 10000
user_ids_reduced = random.sample(list(user_ids), num_users)
print("# de usuarios:", len(user_ids_reduced))

# de usuarios: 10000


Ahora, el número de transacciones será más reducido, si bien lo suficientemente representativo, para el cómputo de las métricas de evaluación del sistema recomendador.

In [5]:
ratings_reduced_df = ratings_df[ratings_df['user_id'].isin(user_ids_reduced)]
print("# de ratings:", ratings_reduced_df.shape[0])

# de ratings: 1053374


Separamos el dataset de ratings en parte de entrenamiento y parte de test, en una proporción del 80% y 20%, respectivamente.

In [6]:
# Creamos los datasets de train (80%) y test (20%)
import sklearn.model_selection as model_selection

train, test = model_selection.train_test_split(ratings_reduced_df, test_size=0.2, random_state=42)
print("# de ratings en train (train + validation):", train.shape[0])
print("# de ratings en test:", test.shape[0])

# de ratings en train (train + validation): 842699
# de ratings en test: 210675


Además, en vista de modelos futuros, el dataset de entrenamiento será parte entrenamiento (70% del global) y parte validación (10% del global). Es fácil comprobar que se debe hacer un split local del dataset de entrenamiento en un 87.5% para entrenamiento *per se* y un 12.5% para validación, consiguiendo los porcentajes globales deseados.

In [7]:
# Creamos los datasets de train (87.5% -> 70% del global) y validación (12.5% -> 10% del global)
train_train, train_valid = model_selection.train_test_split(train, test_size=0.125, random_state=42)
print("# de ratings en train:", train_train.shape[0])
print("# de ratings en validation:", train_valid.shape[0])

# de ratings en train: 737361
# de ratings en validation: 105338


### Entrenamiento del modelo (perfiles de usuario)

Para el modelo, crearemos un dataset de perfiles de usuario que codifique sus preferencias a partir del conjunto de datos de entrenamiento. Los perfiles de usuario estarán basados en los libros que haya puntuado. En concreto, se tendrán las mismas columnas que con los libros: `semantic_sbert`, `semantic_use` y `sentiment`. Pero, en este caso, cada una de las columnas corresponderá a una ponderación calculada de esta manera:

Tomaremos aquellos libros que haya valorado positivamente, es decir, con una puntuación mayor o igual que 0.75. Sea entonces $i_k$ el identificador del k-ésimo libro puntuado por el usuario de ese subconjunto de libros. El vector resultante es el normalizado de $$\frac{\sum_{k}^{}rating(i_k) * \vec{v}_{libro}(i_k)}{\sum_{k}^{}rating(i_k)}$$

In [8]:
# Creamos el dataframe de entrenamiento para el modelo a partir del dataset de train
train_df = pd.DataFrame(train, columns=['user_id', 'book_id', 'rating'])
train_df.sort_values(by=['user_id', 'book_id'], inplace=True)
train_df.reset_index(drop=True, inplace=True)
train_df.head()

,user_id,book_id,rating
0,8,13,0.75
1,8,14,1.00
2,8,28,0.75
3,8,43,0.25
4,8,45,0.50


In [9]:
# Creamos un dataframe con los usuarios que aparecen en el dataset de train
users_df = pd.DataFrame(columns=['user_id'])
users_df['user_id'] = train_df['user_id'].unique()
users_df.sort_values(by=['user_id'], inplace=True)
users_df.reset_index(drop=True, inplace=True)
print("# de usuarios:", users_df.shape[0])

# de usuarios: 10000


In [10]:
# Obtenemos la información de los libros que han sido valorados por los usuarios del dataset de train
books_df = books_df[books_df['book_id'].isin(train_df['book_id'].unique())]
books_df.sort_values(by=['book_id'], inplace=True)
books_df.reset_index(drop=True, inplace=True)
print("# de libros:", books_df.shape[0])

# de libros: 9467


In [11]:
import numpy as np

def create_user_feature(user_id: int, feature: str, train_df: pd.DataFrame, books_df: pd.DataFrame):
    """
    Crea un vector normalizado de la característica dada a partir
    de los libros que al usuario le gustaron (rating >= 0.75). El vector
    es la suma ponderada de los vectores de la característica dada de
    dichos libros, donde los pesos son los ratings de los libros.

    ## Parámetros
    - user_id: ID del usuario.
    - feature: Nombre de la característica (columna) a calcular.
    - train_df: `DataFrame` del dataset de entrenamiento.
    - books_df: `DataFrame` de libros en el dataset de entrenamiento.

    ## Retorna
    Vector normalizado de la característica dada.
    """
    # Ratings positivas del usuario
    user_ratings = train_df[train_df['user_id'] == user_id]
    user_likes = user_ratings[user_ratings['rating'] >= 0.75]
    # Libros que le gustaron al usuario
    user_books = books_df[books_df['book_id'].isin(user_likes['book_id'])]
    feature_array = np.array(user_books[feature])
    # Media ponderada de los vectores de la característica dada de los libros
    user_feature = np.sum([feature_array[i]*np.array(user_likes['rating'])[i] for i in range(len(feature_array))], axis=0)
    total_ratings_sum = np.sum(np.array(user_likes['rating']))
    if total_ratings_sum != 0:
        user_feature = user_feature/total_ratings_sum
    # Se retorna el vector normalizado
    norm = np.linalg.norm(user_feature)
    if norm == 0:
        return user_feature
    return user_feature/norm

In [12]:
# Ejemplo de creación de perfil de usuario (ID 1, característica 'sentiment')
create_user_feature(8, 'sentiment', train_df, books_df)

array([0.28952047, 0.02280815, 0.003686  , 0.02195512, 0.26275696,
       0.00591408, 0.01460672, 0.02984982, 0.00393872, 0.05089228,
       0.01568355, 0.00635603, 0.0054176 , 0.02757305, 0.04054261,
       0.00514268, 0.0040755 , 0.01042422, 0.00743289, 0.00952253,
       0.90845533, 0.0146427 , 0.005234  , 0.08772764, 0.00430895,
       0.0137733 , 0.06895245, 0.0341595 ])

In [13]:
# Característica semantic_sbert de los usuarios
users_df['semantic_sbert'] = users_df['user_id'].apply(lambda x: create_user_feature(x, 'semantic_sbert', train_df, books_df))

In [14]:
# Característica semantic_use de los usuarios
users_df['semantic_use'] = users_df['user_id'].apply(lambda x: create_user_feature(x, 'semantic_use', train_df, books_df))

In [15]:
# Característica sentiment de los usuarios
users_df['sentiment'] = users_df['user_id'].apply(lambda x: create_user_feature(x, 'sentiment', train_df, books_df))

Para posibles usos recurrentes, guardamos el modelo entrenado en almacenamiento local. Realmente se estará guardando un dataset de perfiles de usuario obtenido a partir del 80% del dataset de ratings correspondiente al entrenamiento.

In [16]:
# Ruta de almacenamiento de modelos
models_path = os.path.join(os.getcwd(), "../..", "models")

# Guardamos el modelo KNN entrenado (dataset de usuarios)
users_df.to_pickle(models_path + "/user_profiles.pkl")

Podemos utilizar esta celda para volver a cargar el modelo entrenado, sin necesidad de pasar por el entrenamiento de nuevo.

In [13]:
models_path = os.path.join(os.getcwd(), "../..", "models")
users_df = pd.DataFrame(pd.read_pickle(models_path + "/user_profiles.pkl"))

### Predicciones a partir del modelo

Para hacer las predicciones, necesitamos crear una función de similitud que el modelo utilizará para encontrar los $k$ libros más similares en función del perfil de usuario. Como tanto los libros como los perfiles de usuario cuentan con los mismos atributos (`semantic_sbert`, `semantic_use` y `sentiment`) la función de similitud podrá utilizarse para comparar libros con libros, usuarios con usuarios o libros con usuarios (y viceversa).

La fórmula propuesta es la siguiente: $$S(x, y) = w * S_C(x_{sem}, y_{sem}) + (1 - w) * S_C(x_{sent}, y_{sent}),$$ donde $w \in [0, 1]$ es un peso para la ponderación entre la parte semántica y la parte de sentiment analysis y $S_C$ es la similitud coseno entre los vectores.

In [14]:
# Definimos la función de similitud entre dos objetos
def sem_sent_sim(item1, item2, sem_option='semantic_sbert', sem_w=0.75) -> float:
    """
    Calcula la similitud entre un objeto y otro.
    Los objetos deben tener un vector `'semantic'` y otro `'sentiment'`, 
    previamente normalizados.

    La similitud sigue la siguiente fórmula:

    sim(item1, item2) = sem_w * cos_sim(item1[sem_option], item2[sem_option]) + 
    (1 - sem_w) * cos_sim(item1['sentiment'], item2['sentiment'])

    ## Parámetros:
    - item1: Primer objeto.
    - item2: Segundo objeto.
    - sem_option: Opción de contenido semántico. Por defecto su valor
    es `'semantic_sbert'` (modelo SBERT), pero también puede valer `'semantic_use'`
    (modelo USE).
    - sem_w: Peso del contenido semántico. Por defecto su valor es 0.75.

    ## Retorna:
    La similitud entre ambos objetos.
    """
    # Obtenemos el campo semántico de cada objeto
    sem1 = item1[sem_option]
    sem2 = item2[sem_option]
    # Obtenemos el campo de sentimiento de cada objeto
    sent1 = item1['sentiment']
    sent2 = item2['sentiment']
    # Calculamos la similitud
    return sem_w * np.dot(sem1, sem2) + (1 - sem_w) * np.dot(sent1, sent2)

def sem_sent_dist(item1, item2, sem_option='semantic_sbert', sem_w=0.75) -> float:
    """
    Calcula la distancia entre un objeto y otro.

    Los objetos deben tener un vector 'semantic' y otro 'sentiment', 
    previamente normalizados.

    La similitud sigue la siguiente fórmula:

    dist(item1, item2) = 1 - sim(item1, item2)

    ## Parámetros:
    - item1: Primer objeto.
    - item2: Segundo objeto.
    - sem_option: Opción de contenido semántico. Por defecto su valor
    es `'semantic_sbert'` (modelo SBERT), pero también puede valer `'semantic_use'`
    (modelo USE).
    - sem_w: Peso del contenido semántico. Por defecto su valor es 0.75.

    ## Retorna:
    La distancia entre ambos objetos.
    """
    return 1 - sem_sent_sim(item1, item2, sem_option, sem_w)

In [15]:
def k_nearest(item, train_df: pd.DataFrame, books_df: pd.DataFrame, k=10, sem_option='semantic_sbert', sem_w=0.75):
    """
    Calcula los k vecinos más cercanos a un objeto.

    ## Parámetros:
    - item: Objeto del que queremos obtener los k vecinos más cercanos.
    - train_df: `DataFrame` del dataset de entrenamiento.
    - books_df: `DataFrame` de libros en el dataset de entrenamiento.
    - k: Número de vecinos más cercanos que queremos obtener.
    - sem_option: Opción de contenido semántico. Por defecto su valor
    es `'semantic_sbert'` (modelo SBERT), pero también puede valer `'semantic_use'`
    (modelo USE).
    - sem_w: Peso del contenido semántico. Por defecto su valor es 0.75.

    ## Retorna:
    k tuplas (book_id, similitud) con los k vecinos más cercanos al objeto.
    """
    if 'user_id' in item:
        # Ratings positivas del usuario
        user_ratings = train_df[train_df['user_id'] == item['user_id']]
        user_likes = user_ratings[user_ratings['rating'] >= 0.75]
        # Libros que le gustaron al usuario
        user_books = books_df[books_df['book_id'].isin(user_likes['book_id'])]
        # Eliminamos los libros que ya ha valorado el usuario positivamente
        books_df = books_df[~books_df['book_id'].isin(user_books['book_id'])]
    # Calculamos las distancias entre el objeto y todos los demás
    sim_items = books_df.apply(lambda x: (x['book_id'], sem_sent_sim(item, x, sem_option, sem_w)), axis=1)
    # Obtenemos los k vecinos más cercanos
    nearest = list(sim_items)
    nearest.sort(key=lambda x: x[1], reverse=True)
    return nearest[:k]

Este es un ejemplo para los libros más parecidos a *The Hunger Games* de Suzanne Collins.

In [16]:
hunger_games = books_df[books_df['book_id'] == 1].iloc[0]
k_nearest(hunger_games, books_df, books_df, k=10, sem_w=0.8)

[(1, 0.9999999046325685),
 (20, 0.7180957167741884),
 (17, 0.700777192216677),
 (9709, 0.652521970507263),
 (9713, 0.652521970507263),
 (2533, 0.6492256776360381),
 (12, 0.6162101142787808),
 (6423, 0.6135209588698566),
 (8476, 0.6069897838082223),
 (3318, 0.6055980757300952)]

In [17]:
# Ejemplo de obtención de los 10 libros más cercanos a un usuario (ID 8)
nearest = k_nearest(users_df[users_df['user_id'] == 8].iloc[0], train_df, books_df, k=20, sem_w=0.8)
nearest

[(3134, 0.8115163832462231),
 (2953, 0.784469202129653),
 (4769, 0.7813056304936998),
 (6769, 0.7806696547762818),
 (1123, 0.7793301752215991),
 (2679, 0.7774603923897423),
 (7384, 0.7759015484280399),
 (689, 0.7733627875957638),
 (4106, 0.7704233454650772),
 (7366, 0.7689058575298591),
 (5734, 0.7680697344876575),
 (5738, 0.7677471685578726),
 (9216, 0.7674748400355997),
 (1217, 0.7658872682310269),
 (2663, 0.7653888900117081),
 (3962, 0.7652080251387171),
 (7193, 0.7643296189680527),
 (5587, 0.7616026246820269),
 (8643, 0.7586364812754934),
 (6713, 0.7577003526509288)]

In [18]:
# Calculamos los k libros más similares (de todo el dataset de libros) a cada perfil de usuario (sin contar los ya positivamente valorados)
k = 50
predictions_df = users_df.copy()
predictions_df = predictions_df.drop(columns=['semantic_sbert', 'semantic_use', 'sentiment'])
predictions_df['nearest'] = predictions_df['user_id'].apply(lambda x: k_nearest(users_df[users_df['user_id'] == x].iloc[0], train_df, books_df, k=k, sem_w=1.0))

In [19]:
predictions_df.head()

,user_id,nearest
0,8,"[(2316, 0.7788905501365662), (5936, 0.77398431..."
1,13,"[(3962, 0.8165419101715088), (1198, 0.80972325..."
2,14,"[(5936, 0.8029672503471375), (1198, 0.77596586..."
3,20,"[(2542, 0.8051701188087463), (692, 0.798462748..."
4,28,"[(1198, 0.8236038684844971), (3134, 0.81698650..."


In [20]:
# Transformamos la columna 'nearest' en dos columnas: 'book_id' y 'prediction'
predictions_df = predictions_df.explode('nearest')
predictions_df[['book_id', 'prediction']] = pd.DataFrame(predictions_df['nearest'].tolist(), index=predictions_df.index)
predictions_df = predictions_df.drop('nearest', axis=1)

In [21]:
predictions_df.head()

,user_id,book_id,prediction
0,8,2316,0.778891
0,8,5936,0.773984
0,8,3134,0.769694
0,8,1198,0.763475
0,8,4106,0.760859


### Creación de los ficheros para Elliot

Una vez calculadas todas las predicciones de los libros por usuario a partir del modelo, guardaremos todos los datasets (entrenamiento, validación y test) así como las propias predicciones en formato .tsv, de tal forma que puedan ser utilizados en `Elliot`.

In [22]:
# Ruta de datasets de entrenamiento
training_path = os.path.join(os.getcwd(), "../..", "datasets", "training")
# Guardamos el dataset de predicciones así como los de entrenamiento, test y validación
predictions_df.to_csv(training_path + "/predictions/predictions_cb_100_0.tsv", sep='\t', header=None, index=False)

In [26]:
# Creamos el dataframe del dataset de test (para darle un formato ordenado)
test_df = pd.DataFrame(test, columns=['user_id', 'book_id', 'rating'])
test_df.sort_values(by=['user_id', 'book_id'], inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [27]:
# Creamos el dataframe de entrenamiento 70% (para darle un formato ordenado)
train_train_df = pd.DataFrame(train_train, columns=['user_id', 'book_id', 'rating'])
train_train_df.sort_values(by=['user_id', 'book_id'], inplace=True)
train_train_df.reset_index(drop=True, inplace=True)

In [28]:
# Creamos el dataframe de validación 10% (para darle un formato ordenado)
train_valid_df = pd.DataFrame(train_valid, columns=['user_id', 'book_id', 'rating'])
train_valid_df.sort_values(by=['user_id', 'book_id'], inplace=True)
train_valid_df.reset_index(drop=True, inplace=True)

In [29]:
train_df.to_csv(training_path + "/train_reduced.tsv", sep='\t', header=None, index=False)
test_df.to_csv(training_path + "/test_reduced.tsv", sep='\t', header=None, index=False)
train_train_df.to_csv(training_path + "/train_train_reduced.tsv", sep='\t', header=None, index=False)
train_valid_df.to_csv(training_path + "/train_valid_reduced.tsv", sep='\t', header=None, index=False)

También podemos intentar predecir la nota que daría un usuario a los libros que ya ha valorado a partir del modelo, mediante la métrica de similitud. Adjuntamos una columna `prediction` al dataset total de ratings.

In [91]:
ratings_pred_df = ratings_reduced_df.copy()
ratings_pred_df['prediction'] = ratings_pred_df.apply(lambda x: sem_sent_sim(users_df[users_df['user_id'] == x['user_id']].iloc[0], books_df[books_df['book_id'] == x['book_id']].iloc[0], sem_w=1.0), axis=1)

In [92]:
ratings_pred_df.head()

,user_id,book_id,rating,prediction
619,8,4,0.50,0.605835
620,8,13,0.75,0.819535
621,8,14,1.00,0.917917
622,8,28,0.75,0.679056
623,8,43,0.25,0.300887


In [93]:
del ratings_pred_df['rating']

In [94]:
# Guardamos el dataset de ratings con la predicción
training_path = os.path.join(os.getcwd(), "../..", "datasets", "training")
ratings_pred_df.to_csv(training_path + "/predictions_cf_100_0.tsv", sep='\t', header=None, index=False)